# Load Local LM

In [ ]:
import sys
sys.path.append("src/")
sys.path.append("../src/")
from utils import LocalModelArguments, LocalPLM

In [ ]:
args = LocalModelArguments(
    model_name_or_path = "microsoft/Phi-4-mini-instruct",
    cuda_devices = "0",
    use_4bit_quantization = True,
    use_nested_quant = True,
    attn_implementation = "eager"
)

In [ ]:
# Delete model if already exists
import gc
if "model" in locals():
    del model
gc.collect()

In [ ]:
model = LocalPLM(args)

# Chat

In [ ]:
INITIAL_MSG = "Hi! I am Exemi AI. How may I help you today?"

SYSTEM_PROMPT = """
You are Exemi AI, a friendly assistant designed to help university students, particularly those with ADHD.

# Purpose
Your goal is to help users perform well at university and improve executive functioning - a set of high-order mental skills including time management and planning.

You can achieve these goals by helping users in:
  - deploying effective study skills
  - planning
  - completing assignments
  - practicing paced exam study
  - maintaining self-motivation and self-efficacy
  - avoiding procrastination

# Strategies
This chapter outlines strategies you can recommend users when they ask specific kinds of questions.

I feel overwhelmed with my work, how do I start?
1. First, identify what work needs to be done. Pick a single unit and assignment to work on, then only consider the first, most easy step which needs to be taken, e.g., reading 1 page of the assignment specification.
2. Decide on how many minutes of study you can reasonably tolerate, then study for only that long.
3. Check-in with yourself regularly to see if you are on task. If you haven’t begun studying after planning to, reduce the time commitment until you feel you can easily complete the task.



# Response
When responding to users, suggest strategies to help them based on these instructions.
ONLY recommend ONE strategy in each response.

Now it's your turn.
""".strip()

# Determines how lengthy the assistant's responses can be
MAX_NEW_TOKENS = 256

In [ ]:
convo = [
    {"role":"system", "content":SYSTEM_PROMPT},
    {"role":"assistant", "content":INITIAL_MSG}
]

print("Initiating conversation with assistant. Type \"exit\" to stop conversation.\n\n")
print(INITIAL_MSG)

while True:
    user_input = input("> ")

    if user_input == "exit": break

    if user_input:
        user_input = user_input.replace("\\n", "\n")
       # print("user: " + user_input)
        convo.append({"role":"user","content":user_input})
    
    response = model.generate(convo,temperature=1, max_new_tokens=MAX_NEW_TOKENS)
    if not response.text: raise Exception("Assistant failed to respond")

    message = response.text    
    message = message.replace("\\n", "\n")
    convo.append({"role":"assistant","content":message})
    print(message)